In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu0 is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)


In [3]:
data_dim = 16
timesteps = 8
nb_classes = 10

In [16]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
# generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, nb_classes))

# generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, nb_classes))

In [18]:
model.fit(x_train, y_train,
          batch_size=64, nb_epoch=5,
          validation_data=(x_val, y_val))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s - loss: 11.5198 - acc: 0.1000 - val_loss: 11.5619 - val_acc: 0.0800
Epoch 2/5
1000/1000 [==============================] - 0s - loss: 11.5134 - acc: 0.0930 - val_loss: 11.5576 - val_acc: 0.0800
Epoch 3/5
1000/1000 [==============================] - 0s - loss: 11.5128 - acc: 0.1030 - val_loss: 11.5607 - val_acc: 0.0800
Epoch 4/5
1000/1000 [==============================] - 0s - loss: 11.5119 - acc: 0.1090 - val_loss: 11.5582 - val_acc: 0.0600
Epoch 5/5
1000/1000 [==============================] - 0s - loss: 11.5102 - acc: 0.1190 - val_loss: 11.5588 - val_acc: 0.0600


In [19]:
x_train.shape

(1000, 8, 16)

# BPM

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("/storage/hpc_irheta/bpm_data/Road_Traffic_Fine_Management_Process_labeled.csv", sep=";")
data['matricola'].replace(0.0, 1, inplace=True)
data['Resource'] = data['Resource'].astype(str)
data['article'] = data['article'].astype(str)

In [6]:
len(data)

470893

In [7]:
cat_cols = ['Activity', 'Resource', 'Variant', 'article', 'dismissal', 'matricola', 'notificationType', 'vehicleClass']
numeric_cols = ['amount', 'expense', 'points']

In [8]:
cat_data = pd.get_dummies(data[cat_cols])
dt_final = pd.concat([data[["Case ID", "event_nr"]+numeric_cols], cat_data], axis=1).fillna(0)

In [9]:
dt_final.head()

,Case ID,event_nr,amount,expense,points,matricola,Activity_Add penalty,Activity_Appeal to Judge,Activity_Create Fine,Activity_Insert Date Appeal to Prefecture,...,dismissal_T,dismissal_U,dismissal_V,dismissal_Z,notificationType_C,notificationType_P,vehicleClass_A,vehicleClass_C,vehicleClass_M,vehicleClass_R
0,A1,1.0,35.0,0.0,0.0,0.0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,A1,2.0,0.0,11.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A100,1.0,35.0,0.0,0.0,0.0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,A100,2.0,0.0,11.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A100,3.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
dt_final.event_nr.max()

10.0

In [11]:
grouped = dt_final.groupby("Case ID")

In [12]:
batches = {}
i = 0
for name, group in grouped:
    case_length = len(group)
    if case_length not in batches:
        batches[case_length] = np.zeros((0,case_length,group.shape[1]-2))
    batches[case_length] = np.concatenate([batches[case_length],group.sort_values("event_nr").as_matrix()[:,2:].reshape((1,case_length,group.shape[1]-2))], axis=0)
    if i > 10:
        break
    i += 1

In [13]:
for k, v in batches.items():
    print k, v.shape

1 (2, 1, 490)
2 (1, 2, 490)
4 (2, 4, 490)
5 (6, 5, 490)
6 (1, 6, 490)


In [16]:
batches[5]

array([[[35.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 11.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0],
        [71.5, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0]],

       [[36.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 13.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0],
        [74.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0]],

       [[36.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 13.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0],
        [74.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0]],

       [[21.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 11.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0],
        [42.5, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0]],

       [[36.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 13.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0],
        [74.0, 0.0, 0.0, ..., 0, 0, 0],
        [0.0, 0.0, 0.0, ..., 0, 0, 0]],



In [17]:
X_train = batches[5][:,:-1,:]
y_train = batches[5][:,1:,:]

In [22]:
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [ ]:
lstmsize = 32
dropout = 0.5
optim = 'adadelta'
#nb_epoch = 50
#batch_size = 6

print('Build model...')
model = Sequential()
model.add(LSTM(lstmsize, return_sequences=True, input_shape=(None, X_train.shape[2])))
model.add(Dropout(dropout))
model.add(TimeDistributedDense(y_train.shape[2]))
#model.add(Activation('relu'))

print('Compiling model...')
model.compile(loss='mean_squared_error', optimizer=optim)

print("Training...")
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
#model.fit(X_train, y_train, batch_size=self.batch_size, nb_epoch=self.nepoch, validation_split=0.3) #, callbacks=[early_stopping])
#model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch)
for seq_len, batch in batches.items():
    X_train = batch[:,:-1,:]
    y_train = batch[:,1:,:]
    model.train_on_batch(X_train, y_train)

Build model...
Compiling model...
Training...


In [31]:
predicted = model.predict(X_train)

In [34]:
model.save("bpm_lst.model")

In [37]:
from keras.models import load_model
model2 = load_model("bpm_lst.model")

In [38]:
predicted2 = model2.predict(X_train)